In [1]:
import os
print(os.listdir())


['.ipynb_checkpoints', 'myModel.h5', 'OCVintergrated.ipynb', 'SudokuImage(Unsolvable).jpg', 'SudokuImage2.jpg', 'SudokuSolver.ipynb', 'sudokuSolver.py', 'utils.py', '__pycache__']


In [2]:
from utils import *
import sudokuSolver

In [3]:
#import image
pathImage = "./SudokuImage2.jpg"
heightImg = 450
widthImg = 450

model = initializePredictionModel()

In [4]:
#1 make a threshold
img = cv2.imread(pathImage)
img = cv2.resize(img, (widthImg,heightImg))
imgBlank = np.zeros((heightImg,widthImg, 3), np.uint8)
imgThreshold = preProcess(img)

In [5]:
#find all countours
imgContours = img.copy()
imgBigContour = img.copy()
contours, hierachy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(imgContours,contours, -1, (0,255,0), 3)

array([[[254, 254, 254],
        [254, 254, 254],
        [254, 254, 254],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       [[254, 254, 254],
        [254, 254, 254],
        [254, 254, 254],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       [[254, 254, 254],
        [254, 254, 254],
        [254, 254, 254],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       ...,

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]]

In [6]:
# Find the biggest contour
biggest, maxArea = biggestContour(contours)

if biggest.size != 0:
    biggest = reorder(biggest)  # Reorder points

    # Draw the contour on the image
    cv2.drawContours(imgBigContour, biggest, -1, (0, 0, 255), 25)  # Pass `biggest` inside a list

    # Define the perspective transform points
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [widthImg, 0], [0, heightImg], [widthImg, heightImg]])

    # Get the perspective transformation matrix
    matrix = cv2.getPerspectiveTransform(pts1, pts2)

    # Warp the image using the transformation matrix
    imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))

    # Initialize a blank image for detected digits (make sure imgBlank is defined)
    imgDetectedDigits = imgBlank.copy()  # Make sure imgBlank exists before using it

    # Convert the warped image to grayscale
    imgWarpColored = cv2.cvtColor(imgWarpColored, cv2.COLOR_BGR2GRAY)

In [7]:
#4 split the image to single digit
# Assuming you have functions like splitBoxes, getPrediction, and displayNumbers already defined

# 1. Split the image into individual digit boxes
imageSolvedDigits = imgBlank.copy()
boxes = splitBoxes(imgWarpColored)

# 2. Get predictions from the model
numbers = getPrediction(boxes, model)

# 3. Display numbers on the detected digits image
imgDetectedDigits = displayNumbers(imgDetectedDigits, numbers, color=(255, 0, 255))

numbers = np.asarray(numbers)
posArray = np.where(numbers > 0, 0, 1)
print(posArray)

1/1 [==============================] - 0s 129ms/step
Predicted Class: 2, Probability: 1.0
1/1 [==============================] - 0s 27ms/step
Predicted Class: 0, Probability: 0.1288311779499054
1/1 [==============================] - 0s 23ms/step
Predicted Class: 5, Probability: 0.9999828338623047
1/1 [==============================] - 0s 28ms/step
Predicted Class: 3, Probability: 0.9999382495880127
1/1 [==============================] - 0s 24ms/step
Predicted Class: 0, Probability: 0.12959478795528412
1/1 [==============================] - 0s 27ms/step
Predicted Class: 8, Probability: 1.0
1/1 [==============================] - 0s 24ms/step
Predicted Class: 4, Probability: 1.0
1/1 [==============================] - 0s 28ms/step
Predicted Class: 0, Probability: 0.12950599193572998
1/1 [==============================] - 0s 26ms/step
Predicted Class: 9, Probability: 1.0
1/1 [==============================] - 0s 24ms/step
Predicted Class: 0, Probability: 0.1289006918668747
1/1 [============

In [8]:
# Convert the numbers into a 2D list (9x9 Sudoku board)
board = [numbers[i:i+9] for i in range(0, 81, 9)]

# Ensure that the board is a list of lists (convert any numpy arrays to lists)
board = [row.tolist() for row in board]  # Convert numpy arrays to regular lists

print("Before solving:")
for row in board:
    print(row)

# Now, you can solve the board using the sudokuSolver (make sure solve() method works)
try:
    solved_board = sudokuSolver.solve(board)
except Exception as e:
    print(f"Error while solving: {e}")
    solved_board = None

print("After solving:")
if solved_board:
    for row in solved_board:
        print(row)
else:
    print("The board could not be solved.")
if solved_board:
    flatList = [item for sublist in solved_board for item in sublist]
    solvedNumbers = flatList * posArray  # Assuming posArray is defined elsewhere

    # Now display the solved digits on the image (assuming displayNumbers function works)
    imageSolvedDigits = displayNumbers(imageSolvedDigits, solvedNumbers)
else:
    print("Board could not be solved.")



Before solving:
[2, 0, 5, 3, 0, 8, 4, 0, 9]
[0, 7, 0, 0, 0, 0, 0, 5, 0]
[9, 0, 4, 0, 0, 0, 6, 0, 7]
[5, 0, 0, 0, 4, 0, 0, 0, 2]
[0, 0, 0, 5, 0, 7, 0, 0, 0]
[6, 0, 0, 0, 3, 0, 0, 0, 8]
[4, 0, 6, 0, 0, 0, 8, 0, 1]
[0, 2, 0, 0, 0, 0, 0, 6, 0]
[8, 0, 1, 2, 0, 9, 7, 0, 4]
After solving:
[2, 6, 5, 3, 7, 8, 4, 1, 9]
[1, 7, 8, 4, 9, 6, 2, 5, 3]
[9, 3, 4, 1, 2, 5, 6, 8, 7]
[5, 8, 9, 6, 4, 1, 3, 7, 2]
[3, 4, 2, 5, 8, 7, 1, 9, 6]
[6, 1, 7, 9, 3, 2, 5, 4, 8]
[4, 9, 6, 7, 5, 3, 8, 2, 1]
[7, 2, 3, 8, 1, 4, 9, 6, 5]
[8, 5, 1, 2, 6, 9, 7, 3, 4]


In [ ]:
#show the image
pts2 = np.float32(biggest) # PREPARE POINTS FOR WARP
pts1 =  np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # PREPARE POINTS FOR WARP
matrix = cv2.getPerspectiveTransform(pts1, pts2)  # GER
imgInvWarpColored = img.copy()
imgInvWarpColored = cv2.warpPerspective(imageSolvedDigits, matrix, (widthImg, heightImg))
inv_perspective = cv2.addWeighted(imgInvWarpColored, 1, img, 0.5, 1)
imgDetectedDigits = drawGrid(imgDetectedDigits)
imageSolvedDigits = drawGrid(imageSolvedDigits)
imageArray = ([img,imgThreshold,imgContours, imgBigContour],
            [imgDetectedDigits, imageSolvedDigits,imgInvWarpColored,inv_perspective])
stackedImage = stackImages(imageArray, 1/1.5)
cv2.imshow('Stacked Images', stackedImage)

# Wait for a key press and check if it's the ESC key (key code 27)
while True:
    key = cv2.waitKey(1) & 0xFF  # Mask for 8-bit ASCII compatibility
    if key == 27:  # ESC key
        break

cv2.destroyAllWindows()  # Close all windows

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'boxes' contains the individual digits from the split image
singleBox = boxes[3]  # Change the index to select a different box

# Display the selected box
plt.imshow(singleBox, cmap='gray')  # 'gray' color map since it's likely a grayscale image
plt.title('Single Box')
plt.axis('off')  # Turn off axis for cleaner view
plt.show()